In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow.keras import models,layers

In [2]:
df = pd.read_csv('../data/train.csv')

In [3]:
df.shape

(42000, 785)

In [4]:
labels = df['label']
features = df.drop(['label'],axis=1)

In [5]:
features.shape
features = features.to_numpy(dtype='float32').reshape(-1,28,28,1)
labels = labels.to_numpy(dtype='float32').reshape(-1,1)

In [6]:
features.shape
labels.shape

(42000, 1)

In [8]:
model = models.Sequential()
model.add( layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)) )
model.add(layers.MaxPooling2D((2, 2)))
model.add( layers.Conv2D(16,(3,3),activation='relu',input_shape=(28,28,1)) )
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10))

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
kf = KFold(n_splits=10,shuffle=True,random_state=41)

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                4010      
Total params: 8,954
Trainable params: 8,954
Non-trainable params: 0
____________________________________________________

In [12]:
for train_ind,val_ind in kf.split(features,labels):
    X_train , X_val = features[train_ind] , features[val_ind]
    y_train , y_val = labels[train_ind] , labels[val_ind]
    model.fit(X_train,y_train,batch_size=64,epochs=1,
              validation_data=(X_val,y_val))

Train on 37800 samples, validate on 4200 samples
37800/37800 [==============================] - 27s 719us/sample - loss: 1.1372 - accuracy: 0.8467 - val_loss: 0.2198 - val_accuracy: 0.9410
Train on 37800 samples, validate on 4200 samples
37800/37800 [==============================] - 30s 783us/sample - loss: 0.1632 - accuracy: 0.9537 - val_loss: 0.1346 - val_accuracy: 0.9619
Train on 37800 samples, validate on 4200 samples
37800/37800 [==============================] - 30s 806us/sample - loss: 0.1099 - accuracy: 0.9682 - val_loss: 0.0883 - val_accuracy: 0.9757
Train on 37800 samples, validate on 4200 samples
37800/37800 [==============================] - 29s 758us/sample - loss: 0.0866 - accuracy: 0.9738 - val_loss: 0.0771 - val_accuracy: 0.9745
Train on 37800 samples, validate on 4200 samples
37800/37800 [==============================] - 28s 747us/sample - loss: 0.0686 - accuracy: 0.9792 - val_loss: 0.0743 - val_accuracy: 0.9776
Train on 37800 samples, validate on 4200 samples
37800/

In [13]:
test_df = pd.read_csv('../data/test.csv')

In [16]:
test_features = test_df.to_numpy(dtype='float32').reshape(-1,28,28,1)

In [17]:
output = model.predict(test_features)

In [23]:
output = output.argmax(axis=-1)

In [46]:
data = { 'ImageId':np.arange(1,output.shape[0]+1),'Label':output }

In [47]:
result_df = pd.DataFrame(data,columns=['ImageId','Label'])

In [50]:
result_df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [52]:
result_df.to_csv('../results/my_submission.csv',index=False)